# meassurement of the accuracy 

accuracy is 
mean{ p in dotplot @ i,j  |  i,j in real_structure }

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

# Load data

In [2]:
import rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

data for 'tRNA-asp' is corrupted, ignoring...
ADDRSW 	121
GLYCFN 	205
tRNAphe 	76
RNAsep 	154
ZHCV 	95
Z-CIDGMP-1 	135
adenin 	71
23sRNA 	2904
p564 	155
srRNA 	120
ZGLY 	198
MDLOOP 	80
R009 	1542
5srRNA 	170
TRP5 	202


# train predict

In [3]:
import simushape as sim
from rna_tools.rnaplfold import rnaplfold

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'])
data.pop("23sRNA")
data.pop("R009")
keys = data.keys()[:3]

def get_str(seq):
    for _, data_seq, db in data.values():
        if seq == data_seq:
            return [db]
    print "SHIT"

# acc_calculator needs seq struct rea
predictions = { k: (data[k][1],data[k][2],sim.predict(model,data[k][1], seq_to_db_function=get_str)) for k in keys  }



data for 'tRNA-asp' is corrupted, ignoring...


# get accuracy

In [4]:
from rna_tools.sukosd import sukosd
from rna_tools.rna_accuracy import get_structure_accuracy

acc_shaker = {k: get_structure_accuracy(*predictions[k])  for  k in keys} 


predictions_suko = { k: (data[k][1],data[k][2],sukosd(data[k][2])) for k in keys }

acc_suko = {k: get_structure_accuracy(*predictions_suko[k])  for  k in keys} 

acc_real = {k: get_structure_accuracy(data[k][1],data[k][2],data[k][0])  for  k in keys}


acc_noshape = {k: get_structure_accuracy(data[k][1],data[k][2],None)  for  k in keys}




../rna_tools/rnaplfold.py:31: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if react == None:


# Visualize 

In [5]:
import pandas
import numpy as np
index= ['suko',"shaker",'real',"no_shape"]
data = map(lambda x: [x[k] for k in keys] ,[acc_suko, acc_shaker, acc_real, acc_noshape])
map(lambda x:x.append(np.mean(x)),data)
pandas.DataFrame(data, columns=keys+['mean'], index=index).T

,suko,shaker,real,no_shape
ADDRSW,0.991740,0.992601,0.987444,0.872423
GLYCFN,0.885658,0.957059,0.000000,0.565605
tRNAphe,0.993797,0.993299,0.995362,0.504352
mean,0.957065,0.980986,0.660935,0.647460
